CheckPoint 3(final)

In [ ]:
import numpy as np
from numpy.random import randint, rand

# Number of bits for each chromosome
n_bits = 50

# Number of iterations
n_iter = 100

# Number of candidates in the population
n_pop = 100

# Crossover rate
r_cross = 0.9

# Mutation rate
r_mut = 1.0 / n_bits


n_courses = 15


n_professors = 11

# Number of rooms
n_rooms = 3

# Number of days
n_days = 5

# Time slots in a day
n_slots = 6

# Course details
courses = {
    0: {"name": "SMD", "type": "Theory", "sections": ['A'], "duration": 1, "professors": [0]},
    1: {"name": "SMD", "type": "Theory", "sections": ['B'], "duration": 1, "professors": [1]},

    2: {"name": "AI LAB", "type": "Lab", "sections": ['A'], "duration": 3, "professors": [2]},
    3: {"name": "AI LAB", "type": "Lab", "sections": ['B'], "duration": 3, "professors": [3]},

    4: {"name": "AI", "type": "Theory", "sections": ['A'], "duration": 1, "professors": [0]},
    5: {"name": "AI", "type": "Theory", "sections": ['B'], "duration": 1, "professors": [4]},

    6: {"name": "PDC", "type": "Theory", "sections": ['A'], "duration": 1, "professors": [5]},
    7: {"name": "PDC", "type": "Theory", "sections": ['B'], "duration": 1, "professors": [6]},

    8: {"name": "Numerical", "type": "Theory", "sections": ['A'], "duration": 1, "professors": [7]},
    9: {"name": "Numerical", "type": "Theory", "sections": ['B'], "duration": 1, "professors": [8]},

    10: {"name": "SE", "type": "Theory", "sections": ['A'], "duration": 1, "professors": [9]},
    11: {"name": "SE", "type": "Theory", "sections": ['B'], "duration": 1, "professors": [10]},
}

# Professor details
professors = {
    0: {"name": "Mr Saad Salman", "courses": [0, 4]},
    1: {"name": "Ms. Noor ul Ain", "courses": [1]},
    2: {"name": "Ms. Maira Hashmi", "courses": [2]},
    3: {"name": "Ms. Zille Huma", "courses": [3]},
    4: {"name": "Mr. Ali Zeeshan", "courses": [5]},
    5: {"name": "Mr.Farrukh Bashir", "courses": [6]},
    6: {"name": "Mr. Muhammad Aleem", "courses": [7]},
    7: {"name": "Mr. Imran Ashraf", "courses": [8]},
    8: {"name": "Mr. Almas", "courses": [9]},
    9: {"name": "Ms. Shafaq Riaz", "courses": [10]},
    10: {"name": "Ms. Sidra Khalid", "courses": [11]},
}

# Room details
rooms = {
    0: {"name": "C301", "size": 60, "type": "classroom"},
    1: {"name": "C302", "size": 60, "type": "classroom"},
    #2: {"name": "C303", "size": 60, "type": "classroom"},
   # 3: {"name": "C304", "size": 60, "type": "classroom"},
   # 4: {"name": "C305", "size": 60, "type": "classroom"},
   # 5: {"name": "C306", "size": 60, "type": "classroom"},
  #  6: {"name": "C307", "size": 60, "type": "classroom"},
  #  7: {"name": "C308", "size": 60, "type": "classroom"},
    2: {"name": "Khyber Lab1", "size": 120, "type": "large hall"},
}

#generate the timetable
def generate_timetable():
    """Generates a timetable with valid room assignments."""
    timetable = []
    assigned_slots = set()

    for course_id, course in courses.items():
        for section in course['sections']:
            professor_id = np.random.choice(course['professors'])
            room_id_1, room_id_2 = None, None  # Initialize room IDs
            assigned_rooms = set()
            assigned_days_slots = set()
            last_assigned_day = None

            # Generate details for Theory courses
            if course['type'] == 'Theory':
                for _ in range(2):  # Generate two lectures for theory courses
                    day, slot = generate_valid_day_slot(n_days, assigned_days_slots)
                    available_rooms = get_available_classrooms(rooms)

                    # Find a valid room for the lecture
                    while not available_rooms or any((day, slot, room) in assigned_slots for room in available_rooms):
                        slot = (slot + 1) % n_slots
                        if slot == 0:
                            day = (day + 1) % n_days
                            if day == 0:
                                day = 1  # Avoid day 0
                    room_id = np.random.choice(available_rooms)

                    while (day, slot, room_id) in assigned_slots:
                        room_id = np.random.choice(available_rooms)

                    assigned_rooms.add(room_id)
                    assigned_days_slots.add((day, slot, room_id))
                    assigned_slots.add((day, slot, room_id))

                    # Ensure the second lecture is not on the same or adjacent days
                    if last_assigned_day is not None:
                        while abs(day - last_assigned_day) <= 1:
                            day = (day + 1) % n_days
                            if day == 0:
                                day = 1
                    last_assigned_day = day

                    if room_id_1 is None:
                        room_id_1 = room_id
                        day_1, slot_1 = day, slot
                    else:
                        room_id_2 = room_id_1  # Assign the same room as the first lecture
                        day_2, slot_2 = day, slot

            # Generate details for Lab courses
            elif course['type'] == 'Lab':
                # First lecture
                day_1, slot_1 = generate_valid_day_slot(n_days, assigned_days_slots)
                available_rooms = get_available_large_halls(rooms)

                # Find a valid room for the lab
                while not available_rooms or any((day_1, slot_1, room) in assigned_slots for room in available_rooms):
                    slot_1 = (slot_1 + 1) % n_slots
                    if slot_1 == 0:
                        day_1 = (day_1 + 1) % n_days
                        if day_1 == 0:
                            day_1 = 1  # Avoid day 0
                room_id_1 = np.random.choice(available_rooms)

                while (day_1, slot_1, room_id_1) in assigned_slots:
                    day_1, slot_1 = generate_valid_day_slot(n_days, assigned_days_slots)

                assigned_days_slots.add((day_1, slot_1, room_id_1))
                assigned_slots.add((day_1, slot_1, room_id_1))

                day_2, slot_2 = day_1, (slot_1 + 1) % n_slots
                room_id_2 = room_id_1  # Assign the same room as the first lecture

            # Convert details to binary
            chromosome = format(course_id, '04b')
            chromosome += '1' if course['type'] == 'Lab' else '0'
            chromosome += '00' if section == 'A' else '01'
            chromosome += format(professor_id, '04b')
            chromosome += format(day_1, '03b')
            chromosome += format(slot_1, '03b')
            chromosome += format(room_id_1, '04b')
            chromosome += format(day_2, '03b')
            chromosome += format(slot_2, '03b')
            chromosome += format(room_id_2, '04b')

            timetable.append(chromosome)
    return timetable

# Helper function
def generate_valid_day_slot(n_days, assigned_days_slots):
    """Generates a random day and slot within constraints."""
    while True:
        day = np.random.randint(0, n_days - 1)  #avoid day 5 (index 4)
        slot = np.random.randint(0, n_slots)
        if (day, slot) not in assigned_days_slots:
            return (day, slot) if day < 5 else (0, slot)  #reschedule on Monday if day is 5


def get_available_classrooms(rooms):
    """Returns a list of classroom IDs that are available."""
    return [room_id for room_id, room in rooms.items() if room['type'] == 'classroom']

def get_available_large_halls(rooms):
    """Returns a list of large hall IDs that are available."""
    return [room_id for room_id, room in rooms.items() if room['type'] == 'large hall']

#decode chromosome
def decode_chromosome(chromosome):
    course_id = int(chromosome[:4], 2)
    course_type = 'Lab' if chromosome[4] == '1' else 'Theory'
    section = 'A' if chromosome[5:7] == '00' else 'B'
    professor_id = int(chromosome[7:11], 2)
    day_1 = int(chromosome[11:14], 2)
    slot_1 = int(chromosome[14:17], 2)
    room_1 = int(chromosome[17:21], 2)
    day_2 = int(chromosome[21:24], 2)
    slot_2 = int(chromosome[24:27], 2)
    room_2 = int(chromosome[27:], 2)
    return course_id, course_type, section, professor_id, day_1, slot_1, room_1, day_2, slot_2, room_2

#calculate fitness
def calculate_fitness(timetable):
    clashes = 0
    for i in range(len(timetable)):
        for j in range(len(timetable)):
            if i != j:
                course_id_i, course_type_i, section_i, professor_id_i, day_1_i, slot_1_i, room_1_i, day_2_i, slot_2_i, room_2_i = decode_chromosome(timetable[i])
                course_id_j, course_type_j, section_j, professor_id_j, day_1_j, slot_1_j, room_1_j, day_2_j, slot_2_j, room_2_j = decode_chromosome(timetable[j])
                if professor_id_i == professor_id_j:
                    if day_1_i == day_1_j and slot_1_i == slot_1_j:
                        clashes += 1
                    if day_2_i == day_2_j and slot_2_i == slot_2_j:
                        clashes += 1
                if room_1_i == room_1_j:
                    if day_1_i == day_1_j and slot_1_i == slot_1_j:
                        clashes += 1
                if course_type_i == course_type_j:
                    if day_1_i == day_1_j and slot_1_i == slot_1_j:
                        clashes += 1
    for day in range(n_days):
        for slot in range(n_slots):
            rooms_booked = [0] * n_rooms
            for chromosome in timetable:
                course_id, course_type, section, professor_id, day_1, slot_1, room_1, day_2, slot_2, room_2 = decode_chromosome(chromosome)
                if day_1 == day and slot_1 == slot:
                    rooms_booked[room_1] += 1
                if day_2 == day and slot_2 == slot and room_2 != -1:
                    rooms_booked[room_2] += 1
            for room_count in rooms_booked:
                if room_count > 1:
                    clashes += 1
    return -clashes

#perform selection
def selection(pop, scores, k=3):
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

#perform crossover
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if rand() < r_cross:
        pt = randint(1, len(p1)-2)
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

#perform mutation
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        for j in range(len(bitstring[i])):
            if isinstance(bitstring[i][j], int) and rand() < r_mut:
                bitstring[i][j] = 1 - bitstring[i][j]

#generate the next generation
def create_next_generation(pop, selected, scores):
    children = list()
    for i in range(0, n_pop, 2):
        p1, p2 = selected[i], selected[i+1]
        for c in crossover(p1, p2, r_cross):
            mutation(c, r_mut)
            children.append(c)
    return children

#print timetable in readable format
def print_timetable(timetable):
    for chromosome in timetable:
        course_id, course_type, section, professor_id, day_1, slot_1, room_1, day_2, slot_2, room_2 = decode_chromosome(chromosome)
        course_name = courses[course_id]["name"]
        professor_name = professors[professor_id]["name"]
        if course_type == 'Lab':
            print(f"Course: {course_name}, Type: {course_type}, Section: {section}, Professor: {professor_name}")
            if room_1 in rooms:  # Check if room_1 key exists
                print(f"Lecture: Day: {day_1}, Slot: {slot_1}, Room: {rooms[room_1]['name']}")
        else:
            print(f"Course: {course_name}, Type: {course_type}, Section: {section}, Professor: {professor_name}")
            if room_1 in rooms:  # Check if room_1 key exists
                print(f"Lecture 1: Day: {day_1}, Slot: {slot_1}, Room: {rooms[room_1]['name']}")
            if room_2 != -1 and room_2 in rooms:  # Check if room_2 key exists and value is not -1
                print(f"Lecture 2: Day: {day_2}, Slot: {slot_2}, Room: {rooms[room_2]['name']}")
    print()

def genetic_algorithm(n_iter, n_pop, r_cross, r_mut):
    pop = [generate_timetable() for _ in range(n_pop)]
    best, best_eval = None, float('-inf')
    better_chromosomes = []
    fitness_values = []
    all_chromosomes = []

    for gen in range(n_iter):
        scores = [calculate_fitness(c) for c in pop]
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                better_chromosomes.append((gen, pop[i], scores[i]))
                print(">%d, new best f(%s) = %.3f" % (gen, pop[i], scores[i]))
        selected = [selection(pop, scores) for _ in range(n_pop)]
        pop = create_next_generation(pop, selected, scores)
        fitness_values.append(scores)
        all_chromosomes.append(pop)

    return best, best_eval, better_chromosomes, fitness_values, all_chromosomes

#Print in table table format
def print_actual_timetable(timetable):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    slot_timings = {
        1: "8:30",
        2: "10:00",
        3: "11:30",
        4: "13:00",
        5: "14:30",
        6: "16:00"
    }
    print("Slot Timings:")
    for slot, timing in slot_timings.items():
        print(f"Slot {slot}: {timing}")
    print()

    for day in range(n_days):
        print(f"{days[day]} (Day {day}):")
        for slot in range(1, n_slots + 1):
            print(f"Slot {slot} ({slot_timings[slot]}):")
            for chromosome in timetable:
                course_id, course_type, section, professor_id, day_1, slot_1, room_1, day_2, slot_2, room_2 = decode_chromosome(chromosome)
                course_name = courses[course_id]["name"]
                professor_name = professors[professor_id]["name"]
                if day_1 == day and slot_1 == slot:
                    print(f"    Course: {course_name}, Type: {course_type}, Section: {section}, Professor: {professor_name}")
                    if room_1 in rooms:  # Check if room_1 key exists
                        print(f"        Lecture 1: Room: {rooms[room_1]['name']}")
                if day_2 == day and slot_2 == slot:
                    print(f"    Course: {course_name}, Type: {course_type}, Section: {section}, Professor: {professor_name}")
                    if room_2 in rooms:  # Check if room_2 key exists
                        print(f"        Lecture 2: Room: {rooms[room_2]['name']}")
            print()
        print()



best_timetable, best_fitness, better_chromosomes, fitness_values, all_chromosomes = genetic_algorithm(n_iter, n_pop, r_cross, r_mut)
print("Fitness function values:")
for gen, fitness in enumerate(reversed(fitness_values)):
    print(f"Generation {gen}: Fitness = {max(fitness)}")

#print("\nChromosomes of all generations:")
#for gen, chromosomes in enumerate(all_chromosomes):
#    print(f"Generation {gen}:")
#    for chromosome in chromosomes:
#        print(chromosome)
#    print()

print("\nBest Timetable:")

print_timetable(best_timetable)
print_actual_timetable(best_timetable)


>0, new best f(['0000000000001100000000010110000', '0001001000100001100000100010000', '0010100001000101000100010110010', '0011101001101110100100110000010', '0100000000001101000010010100001', '0101001010000000000010100110001', '0110000010101100100010000010001', '0111001011000001000010101010001', '1000000011100100000010111000001', '1001001100000010100000110110000', '1010000100100101000010110010001', '1011001101001000000011001000001']) = -2.000
>0, new best f(['0000000000000110100010111010001', '0001001000101001100010000100001', '0010100001000101000100010110010', '0011101001100010100100000000010', '0100000000001010000010000110001', '0101001010000000000000100010000', '0110000010100101000000111000000', '0111001011001100100010010000001', '1000000011101001000001000000000', '1001001100001000000010001010001', '1010000100101110100010010100001', '1011001101000100100010110110001']) = -1.000
>0, new best f(['0000000000000010100000100010000', '0001001000101110000000011010000', '001010000100100000010